# COGS 108 - Data Checkpoint

# Names

- Jiay Zhao
- Wenbo Hu
- Xiaotong Zeng
- Yiyun Huang

<a id='research_question'></a>
# Research Question

Is there a statistically significant relationship between the scale (burning area) of wildfire and  climate variables that are associated with global warming such as relative humidity level, temperature and wind speed? Additionally, how can we utilize these variables to predict the next wildfire event in California and the scale of wildfire.

# Dataset(s)

*Fill in your dataset information here*

(Copy this information for each dataset)
- Dataset Name:
- Link to the dataset:
- Number of observations:

1-2 sentences describing each dataset. 

If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.

# Setup

In [2]:
## YOUR CODE HERE

# Data Cleaning

Describe your data cleaning steps here.

In [2]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION

# Project Proposal (updated)

| Meeting Date  | Meeting Time| Completed Before Meeting  | Discuss at Meeting |
|---|---|---|---|
| 1/20  |  1 PM | Read & Think about COGS 108 expectations; brainstorm topics/questions  | Determine best form of communication; Discuss and decide on final project topic; discuss hypothesis; begin background research | 
| 1/26  |  4 PM |  Do background research on topic | Discuss ideal dataset(s) and ethics; draft project proposal | 
| 1/28  |  8 PM  | Edit, finalize, and submit proposal | Search for datasets    |
| 2/3  | 8 PM  | Search for datasets | Discuss wrangling/EDA Plan; Assign group members to lead each specific part |
| 2/10  | 8 PM  | Data checkpoint | Review Data set; Edit wrangling/EDA   |
| 2/17  | 8 PM  | Import & Wrangle Data; EDA | Review/Edit wrangling/EDA; Discuss Analysis Plan   |
| 2/24  | 8 PM  | Finalize wrangling/EDA; Begin Analysis | Discuss/edit Analysis; Submit EDA Checkpoint |
| 3/13  | 8 PM  | Complete analysis; Draft results/conclusion/discussion | Discuss/review/edit full project |
| 3/17  | Before 11:59 PM  | NA | Final check; Turn in Final Project & Group Project Surveys |